In [ ]:
#| default_exp common
from nbdev import *
from nbdev.showdoc import *

## Common

> Functions that you can use quite often

In [ ]:
#| exporti
from collections.abc import Iterable
from collections import namedtuple, deque, defaultdict
from itertools   import chain, combinations, permutations
import string
import re
from typing import Union
import numpy as np
from functools import partial
import functools
from types import FunctionType

In [ ]:
#| export
def copy_func(f):
    "Copy a non-builtin function (NB `copy.copy` does not work for this)"
    if not isinstance(f,FunctionType): return copy(f)
    fn = FunctionType(f.__code__, f.__globals__, f.__name__, f.__defaults__, f.__closure__)
    fn.__dict__.update(f.__dict__)
    return fn

def patch_to(cls, as_prop=False):
    "Decorator: add `f` to `cls`"
    if not isinstance(cls, (tuple,list)): cls=(cls,)
    def _inner(f):
        for c_ in cls:
            nf = copy_func(f)
            # `functools.update_wrapper` when passing patched function to `Pipeline`, so we do it manually
            for o in functools.WRAPPER_ASSIGNMENTS: setattr(nf, o, getattr(f,o))
            nf.__qualname__ = f"{c_.__name__}.{f.__name__}"
            setattr(c_, f.__name__, property(nf) if as_prop else nf)
        return f
    return _inner

def patch(f):
    "Decorator: add `f` to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls)(f)

In [ ]:
class Test():
    def __init__(self):
        pass

In [ ]:
@patch_to(Test)
def hi(self):
    print('hi there')
Test().hi()

hi there


In [ ]:
#| export
def to_int(
    it: Iterable,    # watch out because passing a string '12t' will be ripped into a list [1,2,t]
    intonly=False):  # removes non-ints if set to True
    """
    Returns contents of iterable converted to int if possible
    
    Examples
    --------
    >>> to_int(['-12', 2, 'a'])
    [-12, 2, 'a']
    """

    if not it:
        print('empty line, returning []')
        return []
    if isinstance(it,str):
        print('watch out string will be converted into list of characters and single digit ints')
    if isinstance(it[0],list):
        return list(to_int(l) for l in it)
    if isinstance(it[0],tuple):
        return tuple(to_int(l) for l in it)

    out = []
    for i in it:
        try:
            out.append(int(i))
        except ValueError:
            if not intonly:
                out.append(i)
    if isinstance(it,tuple): return tuple(out)
    else: return list(out)

In [ ]:
assert to_int(["12",2,'a']) == [12, 2, 'a']
assert to_int(["12",2,'a'], intonly=True) == [12, 2]
assert to_int([[[1],[-2,3]],[4,5,6]]) == [[[1], [-2, 3]], [4, 5, 6]]
assert to_int('bla 202') == ['b', 'l', 'a', ' ', 2, 0, 2]
assert to_int(None) == []
assert to_int('') == []
assert to_int([]) == []

watch out string will be converted into list of characters and single digit ints
empty line, returning []
empty line, returning []
empty line, returning []


In [ ]:
#| export
def ints(text: str) -> tuple[int]:
    """
    Return a tuple of all the integers in a string, discards everything else
    """
    return tuple(map(int, re.findall('-?[0-9]+', text)))

In [ ]:
assert ints('blabla202test-20') == (202,-20)

In [ ]:
#| export
def flatten(x:Iterable):
    def _flatten(x):
        for item in x:
            if isinstance(item,Iterable) and not isinstance(item, str):
                yield from _flatten(item)
            else:
                yield item
    """ 
    recursive flattens the input. Returns a list
    """
    return list(_flatten(x))



In [ ]:
assert flatten([1,2,4,[99,33,[22,11]], 'f']) == [1, 2, 4, 99, 33, 22, 11, 'f']
assert flatten([[[1],[2,3]],[4,5,6]]) == [1, 2, 3, 4, 5, 6]

In [ ]:
#| export
def zippify(iterable, len=2, cat=False):
    """
        Zips an iterable with arbitrary length pieces
        
        e.g. to create a moving window with len n
        
        Example:
        zippify('abcd',2, cat=False)
        --> [('a', 'b'), ('b', 'c'), ('c', 'd')]

        If cat = True, joins the moving windows together
        zippify('abcd',2, cat=True)
        --> ['ab', 'bc', 'cd']
    """
    iterable_collection = [iterable[i:] for i in range(len)]
    res = list(zip(*iterable_collection))
    return [''.join(r) for r in res] if cat else res

In [ ]:
assert zippify('abcd',2, cat=True) == ['ab', 'bc', 'cd']
assert zippify('abcd',2, cat=False) == [('a', 'b'), ('b', 'c'), ('c', 'd')]
assert zippify('abcde',3) == [('a', 'b', 'c'), ('b', 'c', 'd'), ('c', 'd', 'e')]

In [ ]:
#| export
def multidict(items: Iterable[tuple]) -> dict:
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for key, val in items:
        result[key].append(val)
    return result

In [ ]:
tuples = [(2, 'a'),(2, 'b'),(4, 'a')]
dict(tuples), multidict(tuples)

({2: 'b', 4: 'a'}, defaultdict(list, {2: ['a', 'b'], 4: ['a']}))

In [ ]:
#| export
def rev(d:dict) -> dict:
    "Reverses keys and values. Make sure the value is hashable"
    return {v:k for k,v in d.items()}

In [ ]:
a = {(0,0):'f'}
a |= rev(a)
assert a == {(0, 0): 'f', 'f': (0, 0)}

## Peter Norvig AoC helper functions

> All of these are taken from github of Peter Norvig
> https://github.com/norvig/pytudes/blob/master/ipynb/Advent-2020.ipynb

In [ ]:
#| export
def data(filename='input', parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'{filename}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [ ]:
#| export
def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

In [ ]:
assert quantify(['a','11',11,22], pred=lambda x: isinstance(x, int)) == 2
assert quantify([1,2,3,4,5,6,7], lambda x: x==5) == 1
assert quantify([1,2,3,4], lambda x: x<3) == 2

In [ ]:
#| export
def atom(text: str) -> Union[float, int, str]:
    "Parse text into a single float or int or str."
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text


In [ ]:
atom('11') == 11

True

In [ ]:
#| export
def atoms(text: str, ignore=r'', sep=None) -> tuple[Union[int, str]]:
    "Parse text into atoms (numbers or strs), possibly ignoring a regex."
    if ignore:
        text = re.sub(ignore, '', text)
    return tuple(map(atom, text.split(sep)))


('abc', 111, 'def')

In [ ]:
atoms('abc 111 def')

In [ ]:
#| export
def list_atoms(inp: list):
    return tuple(map(atom, inp))


In [ ]:
assert list_atoms(['1', '3.2', 'a', 1])

In [ ]:
#| export 
def list_multiply(a:Iterable,b:Iterable)->list:
    """
        Multiplies two iterables elementwise

        list_multiply([1,2,3],[2,3,4]) -> [2, 6, 12]
    """
    return (np.array(a)*np.array(b)).tolist()


In [ ]:
assert list_multiply([1,2,3],[2,3,4]) == [2, 6, 12]
assert list_multiply((1,2,3),(2,3,4)) == [2, 6, 12]
assert list_multiply([1,2,3],(2,3,4)) == [2, 6, 12]

In [ ]:
#| export
def mapt(fn, *args):
    "map(fn, *args) and return the result as a tuple."
    return tuple(map(fn, *args))
mapt(lambda x: x+'z', list('abcde'))

('az', 'bz', 'cz', 'dz', 'ez')

In [ ]:
#| export 
cat = ''.join
chainit = chain.from_iterable